In [1]:
import pandas as pd
import os 
import random
import requests
import datetime
import gc 

In [2]:
os.getcwd()

'C:\\Users\\User\\@Code-ML\\[Project]-ESG_responsible_investment\\work\\notebook\\ETL'

### Getting ticker list from data 

In [4]:
snp500_df = pd.read_csv("../../data/transform/dim_snp_esg_full_16-Jan-2022.csv")
display(snp500_df.shape)
snp500_df.head()

(507, 5)

,ticker,company_name,gics_sector,gics_sub_industry,is_esg
0,A,Agilent Technologies,Health Care,Health Care Equipment,1
1,AAL,American Airlines Group,Industrials,Airlines,1
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,0
3,AAPL,Apple,Information Technology,"Technology Hardware, Storage & Peripherals",1
4,ABBV,AbbVie,Health Care,Pharmaceuticals,1


In [161]:
snp500_list = snp500_df['ticker'].tolist()
display(len(snp500_list))
random.sample(snp500_list, 10)

507

['PLD', 'SLB', 'MDLZ', 'GPN', 'AVY', 'JNPR', 'ITW', 'CCI', 'HSY', 'LLY']

### Testing the API 

In [113]:
def financial_data_scrapper_test(one_ticker):
    error_tickers = []
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"
    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': "0d6b57ef54msh4015eb34a7c83a7p14538ejsn2cf132bdfe8b"
        }
    scraped_df = pd.DataFrame()

    querystring = {"symbol":one_ticker,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    data_row = pd.json_normalize(response.json())
    scraped_df = pd.concat([scraped_df, data_row], axis=0).reset_index(drop=True)
    return scraped_df

In [114]:
df2 = financial_data_scrapper_test("AAPL")

### Get data 
Using Rapid API to get data. API limit rate is 500 calls.  

Wonder why append doesnt work.. But pd.concat works fine.. 

In [147]:

def financial_data_scrapper(dl_list):
    error_tickers = []
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"
    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': "1a87b90e75msh7c6537354938f02p191c4cjsna0993a54a78a"
        }
    scraped_df = pd.DataFrame()
    marketcap_df = pd.DataFrame()
    marketcap_tuple_list = []

    for ticker in dl_list: 
        try:
            querystring = {"symbol":ticker,"region":"US"}
            response = requests.request("GET", url, headers=headers, params=querystring)
#             data_row = pd.json_normalize(response.json())
#             scraped_df.append(data_row)
            data_row = pd.json_normalize(response.json())
            scraped_df = pd.concat([scraped_df, data_row], axis=0).reset_index(drop=True)
            print(f'Successfully processed {ticker} at {datetime.datetime.now() }!')
            print(f'DataFrame size is {scraped_df.shape}.\n')
            
            marketcap_tuple = (scraped_df['symbol'].iloc[-1], scraped_df['price.marketCap.raw'].iloc[-1] )                
            print("Marketcap: ",marketcap_tuple)
            marketcap_tuple_list.append(marketcap_tuple)
            
        except KeyError:
            error_tickers.append(ticker)
            print(f'This ticker {ticker} is not found.\n')
            continue
    return scraped_df, marketcap_tuple_list

In [148]:
#Getting the reamaining data 
second_scraped_df, second_marketcap_tuple_list = financial_data_scrapper(snp500_list[500:])         

Successfully processed XRAY at 2022-01-17 16:54:33.816570!
DataFrame size is (1, 449).

Marketcap:  ('XRAY', 11791662080)
Successfully processed XYL at 2022-01-17 16:54:36.125030!
DataFrame size is (2, 449).

Marketcap:  ('XYL', 20113448960)
Successfully processed YUM at 2022-01-17 16:54:38.493570!
DataFrame size is (3, 449).

Marketcap:  ('YUM', 37245476864)
Successfully processed ZBH at 2022-01-17 16:54:40.901587!
DataFrame size is (4, 449).

Marketcap:  ('ZBH', 26610700288)
Successfully processed ZBRA at 2022-01-17 16:54:43.218884!
DataFrame size is (5, 450).

Marketcap:  ('ZBRA', 28217112576)
Successfully processed ZION at 2022-01-17 16:54:45.825517!
DataFrame size is (6, 450).

Marketcap:  ('ZION', 11054111744)
Successfully processed ZTS at 2022-01-17 16:54:48.226250!
DataFrame size is (7, 450).

Marketcap:  ('ZTS', 97549115392)


### Full scraped data

In [102]:
first_scraped_df = scraped_df[:500]

In [154]:
full_scraped_df = pd.concat([first_scraped_df, second_scraped_df])
#full_scraped_df.to_csv('full_financial_details.csv')

### Marketcap data 

In [ ]:
first_marketcap_tuple_list = marketcap_tuple_list[:500]

In [159]:
full_marketcap_tuple_list = first_marketcap_tuple_list +second_marketcap_tuple_list
df_marketcap = pd.DataFrame(full_marketcap_tuple_list, columns =['Ticker', 'Marketcap'])
display(df_marketcap.shape)
df_marketcap.head()

(507, 2)

,Ticker,Marketcap
0,A,4.369350e+10
1,AAL,1.197253e+10
2,AAP,1.494031e+10
3,AAPL,2.826995e+12
4,ABBV,2.402018e+11


In [160]:
#df_marketcap.to_csv('marketcap_data.csv')